In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize 
# load nltk's SnowballStemmer as variabled 'stemmer'
from nltk.stem.snowball import SnowballStemmer
import re 

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import json
wordnet_lemmatizer = WordNetLemmatizer()
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from sklearn import metrics
from scipy.spatial.distance import cdist

In [2]:
import nltk
from nltk.corpus import stopwords

nltk.download('punkt')
stop_words = set(stopwords.words('english')) 

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\zxy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
stop_words=list(set(stop_words))

In [4]:
arti=pd.read_csv('nyt-articles-2020.csv')
len(arti)

16787

In [5]:
com=pd.read_csv('nyt-comments-2020.csv',low_memory=False)

In [6]:
com.columns

Index(['commentID', 'status', 'commentSequence', 'userID', 'userDisplayName',
       'userLocation', 'userTitle', 'commentBody', 'createDate', 'updateDate',
       'approveDate', 'recommendations', 'replyCount', 'editorsSelection',
       'parentID', 'parentUserDisplayName', 'depth', 'commentType', 'trusted',
       'recommendedFlag', 'permID', 'isAnonymous', 'articleID'],
      dtype='object')

In [7]:
artiCom=arti.merge(com, how='left',left_on='uniqueID',right_on='articleID')

In [8]:
#######clean comments for Science article#########
with open('contraction_map.json', 'r') as fp:
    contraction_map = json.load(fp) 
punc='"#$%&\'()*+,-./:;<=>@[\\]^_`{|}~\n?!'
stopwords=list(set(stop_words))
def clean(sentence):
    s=' '.join((contraction_map.get(word, word) for word in sentence.split(' ')))
    s2=''.join([char for char in s if char not in punc])
    lem=[wordnet_lemmatizer.lemmatize(word, pos="v") for word in s2.split(' ')]
    wordlist=[word for word in lem if word.lower() not in stopwords and word.isalpha()]# word list
    a=' '.join(wordlist)
    return a

In [9]:
artiCom.columns

Index(['newsdesk', 'section', 'subsection', 'material', 'headline', 'abstract',
       'keywords', 'word_count', 'pub_date', 'n_comments', 'uniqueID',
       'commentID', 'status', 'commentSequence', 'userID', 'userDisplayName',
       'userLocation', 'userTitle', 'commentBody', 'createDate', 'updateDate',
       'approveDate', 'recommendations', 'replyCount', 'editorsSelection',
       'parentID', 'parentUserDisplayName', 'depth', 'commentType', 'trusted',
       'recommendedFlag', 'permID', 'isAnonymous', 'articleID'],
      dtype='object')

In [10]:
###### clean comments#######
artiCom['clean_com']=artiCom.commentBody.apply(lambda x: clean(x))

In [11]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [12]:
vader = SentimentIntensityAnalyzer()

In [13]:
def detect_vader_comp(text):
    return vader.polarity_scores(text)['compound']

In [14]:
artiCom['com_compound'] = artiCom.clean_com.apply(detect_vader_comp)

In [15]:
artiCom.head()

,newsdesk,section,subsection,material,headline,abstract,keywords,word_count,pub_date,n_comments,...,parentUserDisplayName,depth,commentType,trusted,recommendedFlag,permID,isAnonymous,articleID,clean_com,com_compound
0,Editorial,Opinion,NaN,Editorial,Protect Veterans From Fraud,Congress could do much more to protect America...,"['Veterans', 'For-Profit Schools', 'Financial ...",680,2020-01-01 00:18:54+00:00,186,...,NaN,1,comment,0,0,104387472,False,nyt://article/69a7090b-9f36-569e-b5ab-b0ba5bb3...,something think fraudulent vet subject use VA ...,0.4939
1,Editorial,Opinion,NaN,Editorial,Protect Veterans From Fraud,Congress could do much more to protect America...,"['Veterans', 'For-Profit Schools', 'Financial ...",680,2020-01-01 00:18:54+00:00,186,...,magicisnotreal,2,userReply,0,0,104387873,False,nyt://article/69a7090b-9f36-569e-b5ab-b0ba5bb3...,magicisnotreal use VA loan option four time ev...,0.8479
2,Editorial,Opinion,NaN,Editorial,Protect Veterans From Fraud,Congress could do much more to protect America...,"['Veterans', 'For-Profit Schools', 'Financial ...",680,2020-01-01 00:18:54+00:00,186,...,magicisnotreal,2,userReply,0,0,104387976,False,nyt://article/69a7090b-9f36-569e-b5ab-b0ba5bb3...,magiWhy would someone take VA loan conventiona...,0.5859
3,Editorial,Opinion,NaN,Editorial,Protect Veterans From Fraud,Congress could do much more to protect America...,"['Veterans', 'For-Profit Schools', 'Financial ...",680,2020-01-01 00:18:54+00:00,186,...,magicisnotreal,2,userReply,0,0,104390628,False,nyt://article/69a7090b-9f36-569e-b5ab-b0ba5bb3...,JDOut Alabama PNW try steal home HELOC scam au...,0.7754
4,Editorial,Opinion,NaN,Editorial,Protect Veterans From Fraud,Congress could do much more to protect America...,"['Veterans', 'For-Profit Schools', 'Financial ...",680,2020-01-01 00:18:54+00:00,186,...,magicisnotreal,2,userReply,0,0,104391463,False,nyt://article/69a7090b-9f36-569e-b5ab-b0ba5bb3...,magicisnotreal guess doubt credit union make s...,0.9359


In [16]:
artiCom.to_csv('artiCom_sentimentScore.csv')

In [26]:
comScore_opinion=artiCom[artiCom['section']=='Opinion']

In [27]:
comScore_world=artiCom[artiCom['section']=='World']

In [28]:
comScore_US=artiCom[artiCom['section']=='U.S.']

In [29]:
comScore_world.to_csv('comScore_world.csv')

In [30]:
comScore_opinion.to_csv('comScore_opinion.csv')

In [31]:
comScore_US.to_csv('comScore_US.csv')

In [18]:
#try_=artiCom[0:100]

In [21]:
#try_[['articleID','com_compound']].groupby('articleID').var()

,com_compound
articleID,
nyt://article/69a7090b-9f36-569e-b5ab-b0ba5bb3ccbd,0.348242


In [22]:
sentiment_diversity=artiCom[['articleID','com_compound']].groupby('articleID').var()

In [23]:
sentiment_diversity.head()

,com_compound
articleID,
nyt://article/00036db7-8494-5141-b0ac-414118caabee,0.289364
nyt://article/00045477-4bf1-59d5-874f-b5b61bbcfb36,0.091655
nyt://article/00074f13-bb5f-57c0-80e2-df0921cc62eb,0.239837
nyt://article/00168469-d42e-5a6f-9c81-90e3306c6c85,0.319145
nyt://article/00170d27-c2ca-5d3a-8c05-f27721c3c45f,0.333694


In [24]:
sentiment_diversity.to_csv('article_sentimentDiversity.csv')